# MCELOM

## ***Monte-Carlo Elo Madness***

### Monte-Carlo Simulation - Repeatedly repeating a process with a bit of randomness

*aka simulating a tournament*

### Elo - A simple rating system where win probability can be calculated from rankings

*aka a way to predict the winner for each game*

### Madness - It's March

*I'm not explaining this one further*

## Load Libraries

In [148]:
import pandas as pd
import openpyxl
from random import randint

## Load DataFrames

In [151]:
df320 = pd.read_excel("data/538Data.xlsx", sheet_name="320")
df319 = pd.read_excel("data/538Data.xlsx", sheet_name="319")
df317 = pd.read_excel("data/538Data.xlsx", sheet_name="317")

teamgame_df = pd.read_excel("data/BracketStructure.xlsx", sheet_name="TeamGame")
game_df = pd.read_excel("data/BracketStructure.xlsx", sheet_name="Game")

## Define Functions

In [144]:
def calc_win_chance(df, team1_id, team2_id):
    team1_rating = df[df.team_slot == team1_id].team_rating.values[0]
    team2_rating = df[df.team_slot == team2_id].team_rating.values[0]

    chance = 1.0/(1.0 + 10.0**(-(team1_rating - team2_rating) * 30.464 / 400.0))

    return chance

def pick_winner(teamdf, gamedf, gameid):
    team1 = gamedf[gamedf.game_id == gameid].team1.values[0]
    team2 = gamedf[gamedf.game_id == gameid].team2.values[0]

    threshold = calc_win_chance(teamdf, team1, team2)

    rand_outcome = randint(0,1001)/1000.0

    return team1 if rand_outcome <= threshold else team2

def advance_winner(gamedf, gameid, winner):
    if gamedf[gamedf.game_id == gameid].team1.values[0] != winner and gamedf[gamedf.game_id == gameid].team2.values[0] != winner:
        raise

    i = game_df.index[game_df['game_id'] == gameid].tolist()[0]
    nextgame = game_df['goto_id'][i]
    game_df['winner'][i] = winner

    ni = game_df.index[game_df['game_id'] == nextgame].tolist()[0]

    if gameid == game_df['team1_fromgame'][ni]:
        game_df['team1'][ni] = winner
    elif gameid == game_df['team2_fromgame'][ni]:
        game_df['team2'][ni] = winner
    else:
        raise
    print(gamedf[gamedf.game_id == gameid])

def simulate_games(gamesdf, teamsdf):
    rounds = gamesdf.round.unique()


   game_id goto_id team1_fromgame team2_fromgame  team1  team2  winner
32     b31     c16            BYE            BYE    120    122     122


In [ ]:
#calc_win_chance(df317, 120, 122)
#for i in range(10):
#    print(pick_winner(df317, game_df, "b31"))

game = "b31"
winning_team = pick_winner(df317, game_df, game)
advance_winner(game_df, game, winning_team)

In [145]:
game_df.loc[32]

game_id           b31
goto_id           c16
team1_fromgame    BYE
team2_fromgame    BYE
team1             120
team2             122
winner            122
Name: 32, dtype: object

In [146]:
game_df.loc[15]

game_id           c16
goto_id           d08
team1_fromgame    b31
team2_fromgame    b32
team1             122
team2              -1
winner             -1
Name: 15, dtype: object

In [109]:
x = game_df.loc[game_df.game_id == "b31"].index

AttributeError: 'Int64Index' object has no attribute 'value'

In [168]:
roundsnum = game_df["round"].unique()

roundsnum = roundsnum

roundsnum

array([7, 6, 5, 4, 3, 2, 1], dtype=int64)

In [154]:
game_df.round

<bound method DataFrame.round of    game_id goto_id team1_fromgame team2_fromgame  team1  team2  winner  round
0      g01   Champ            f01            f02     -1     -1      -1      7
1      f02     g01            e03            e04     -1     -1      -1      6
2      f01     g01            e01            e02     -1     -1      -1      6
3      e04     f02            d07            d08     -1     -1      -1      5
4      e03     f02            d05            d06     -1     -1      -1      5
..     ...     ...            ...            ...    ...    ...     ...    ...
62     b01     c01            BYE            a01      0     -1      -1      2
63     a04     b13            BYE            BYE     50     51      -1      1
64     a03     b09            BYE            BYE     34     35      -1      1
65     a02     b05            BYE            BYE     18     19      -1      1
66     a01     b01            BYE            BYE      2      3      -1      1

[67 rows x 8 columns]>